<a href="https://colab.research.google.com/github/aakash0017/Machine-Translation-ISWLT/blob/main/amazon_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/aakash0017/Machine-Translation-ISWLT

Cloning into 'Machine-Translation-ISWLT'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (122/122), done.
remote: Total 165 (delta 89), reused 85 (delta 37), pack-reused 0
Receiving objects: 100% (165/165), 27.52 MiB | 17.27 MiB/s, done.
Resolving deltas: 100% (89/89), done.


In [ ]:
%pwd

'/content/Machine-Translation-ISWLT/evaluation/predictions/fr'

In [ ]:
%cd /content/Machine-Translation-ISWLT/evaluation/predictions/de

/content/Machine-Translation-ISWLT/evaluation/predictions/de


In [ ]:
!git clone https://github.com/amazon-research/isometric-slt

Cloning into 'isometric-slt'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 21 (delta 1), reused 17 (delta 0), pack-reused 0
Unpacking objects: 100% (21/21), done.


In [ ]:
%cd isometric-slt/scripts

/content/Machine-Translation-ISWLT/evaluation/predictions/de/isometric-slt/scripts


In [ ]:
!chmod 744 setup-env.sh
! ./setup-env.sh

In [ ]:
!chmod 744 compute_isometric_slt_stat.sh 

In [ ]:

! ./compute_isometric_slt_stat.sh de /content/Machine-Translation-ISWLT/evaluation/predictions/de/test.en /content/Machine-Translation-ISWLT/evaluation/predictions/de/test_mt_pred1.de /content/Machine-Translation-ISWLT/evaluation/predictions/de/test.de

Computing stat for:
==> /content/Machine-Translation-ISWLT/evaluation/predictions/de/test.en <==
Back in New York, I am the head of development for a non-profit called Robin Hood.
When I'm not fighting poverty, I'm fighting fires as the assistant captain of a volunteer fire company.

==> /content/Machine-Translation-ISWLT/evaluation/predictions/de/test_mt_pred1.de <==
Zurück in New York bin ich der Leiter der Entwicklung für einen gemeinnützigen namens Robin Hood.
Wenn ich nicht gegen die Armut bin, werde ich als Assistentin einer Freiwilligen-Brände

==> /content/Machine-Translation-ISWLT/evaluation/predictions/de/test.de <==
Zu Hause in New York, bin ich Chef der Entwicklungsabteilung einer gemeinnützigen Organisation namens Robin Hood.
Wenn ich nicht die Armut bekämpfe, bekämpfe ich als Gehilfe eines Feuerwehr-Hauptmanns bei einem freiwilligen Löschzug das Feuer.
Downloading: 100% 52.0/52.0 [00:00<00:00, 57.1kB/s]
Downloading: 100% 792/792 [00:00<00:00, 533kB/s]
Downloading: 100% 87